In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_auc_score,
    mean_squared_error,
    r2_score,
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import KNNImputer

from skbio.diversity import alpha_diversity, beta_diversity
from skbio.stats.ordination import pcoa
from skbio.diversity.alpha import shannon
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr, pearsonr
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [2]:
# Dataset 2
data = pd.read_csv('/Users/vbuttros/Library/CloudStorage/OneDrive-Personal/University/Microbiology - DSc/Colaborations/Buttros, VH/R_solanii_ANN/Treated_data/dataset2_script_files/data.csv')

otu_table_16S = pd.read_csv(
    '/Users/vbuttros/Library/CloudStorage/OneDrive-Personal/University/Microbiology - DSc/Colaborations/Buttros, VH/R_solanii_ANN/Treated_data/dataset2_script_files/OTU_table_16S.csv',
    header=0,  # Use the first row as headers
    index_col=0  # Set the first column as the index
)

otu_table_ITS = pd.read_csv(
    '/Users/vbuttros/Library/CloudStorage/OneDrive-Personal/University/Microbiology - DSc/Colaborations/Buttros, VH/R_solanii_ANN/Treated_data/dataset2_script_files/OTU_table_ITS.csv',
    header=0,  # Use the first row as headers
    index_col=0  # Set the first column as the index
)

equivalence_table = pd.read_csv('/Users/vbuttros/Library/CloudStorage/OneDrive-Personal/University/Microbiology - DSc/Colaborations/Buttros, VH/R_solanii_ANN/Treated_data/dataset2_script_files/eq_table.csv')

In [3]:
# Merge Y_rs.dis with otu_table_16S
sev_16S = otu_table_16S.merge(data[['Sample_ID', 'Y_rs.dis']], on='Sample_ID', how='inner')
sev_16S.set_index(sev_16S.columns[0], inplace=True)  # Set column 0 as the index

# Merge Y_rs.dis with otu_table_ITS
sev_ITS = otu_table_ITS.merge(data[['Sample_ID', 'Y_rs.dis']], on='Sample_ID', how='inner')
sev_ITS.set_index(sev_ITS.columns[0], inplace=True)  # Set column 0 as the index

In [4]:
def perform_regression(data, otu_type, n_estimators=100, random_state=42):
    """
    Perform Random Forest regression to calculate feature importance, prioritizing OTUs that reduce the response.

    Parameters:
        data (pd.DataFrame): DataFrame containing predictors and response variable.
        otu_type (str): Type of OTU ('fungal' or 'bacterial') to reflect in output name.
        n_estimators (int): Number of trees in the random forest (default: 100).
        random_state (int): Random seed for reproducibility (default: 42).

    Returns:
        pd.DataFrame: A DataFrame containing OTU and Feature Importance.
    """
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestRegressor

    # Normalize all columns to a 0-1 range
    scaler = MinMaxScaler()
    data_normalized = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)

    # Extract response variable (Y_rs.dis) and predictors (all other columns)
    response = data_normalized['Y_rs.dis']
    predictors = data_normalized.drop(columns=['Y_rs.dis'])

    # Apply inverse weighting to the response to prioritize smaller values
    response_weighted = 1 / (response + 1e-6)  # Adding a small constant to avoid division by zero

    # Fit Random Forest regressor using weighted response
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state, n_jobs=-1)
    rf_model.fit(predictors, response_weighted)

    # Use built-in feature importance
    feature_importance = rf_model.feature_importances_

    # Create a DataFrame for the results
    importance_results = pd.DataFrame({
        'OTU': predictors.columns,
        'Feature_Importance': feature_importance
    })

    # Sort by importance
    importance_results = importance_results.sort_values(by='Feature_Importance', ascending=False)

    # Name the output DataFrame based on OTU type
    importance_results.name = f"{otu_type}_rf_reg"

    return importance_results

In [5]:
reg_sev_ITS = perform_regression(sev_ITS, 'fungal')
reg_sev_16S = perform_regression(sev_16S, 'bacterial')

In [6]:
def reg_proc(feature_importance_results):
    """
    Process feature importance results by normalizing to a 0-1 scale.

    Parameters:
        feature_importance_results (pd.DataFrame): DataFrame containing 'OTU' and 'Feature_Importance'.
    
    Returns:
        pd.DataFrame: Updated DataFrame with a new column 'Normalized_Importance'.
    """
    # Normalize the feature importance to a 0-1 range
    min_val = feature_importance_results['Feature_Importance'].min()
    max_val = feature_importance_results['Feature_Importance'].max()
    
    feature_importance_results['Normalized_Importance'] = (
        (feature_importance_results['Feature_Importance'] - min_val) / 
        (max_val - min_val)
    )

    return feature_importance_results

In [7]:
reg_sev_16S = reg_proc(reg_sev_16S)
reg_sev_ITS = reg_proc(reg_sev_ITS)

In [8]:
def bcon_index(feature_importance_results, abundance_table, otu_type, operation='*', amplify_factor=0.1, final_amplification=0.1):
    """
    Calculate the Biocontrol Index (BCI) with enhanced emphasis on its predictive importance for disease severity.

    Parameters:
        feature_importance_results (pd.DataFrame): DataFrame containing OTUs and their 'Normalized_Importance'.
        abundance_table (pd.DataFrame): Abundance table with OTUs as columns and Sample_ID as index.
        otu_type (str): Type of OTU ('fungal' or 'bacterial') to name the output column.
        operation (str): Operation to apply for BCI calculation ('*', '2', '3', etc.).
        amplify_factor (int): Exponent to amplify the effect of Normalized Importance.
        final_amplification (float): Factor to amplify final BCI values for better interpretability.

    Returns:
        pd.DataFrame: DataFrame with Sample_ID as rows and BCI values as a column.
    """

    # Ensure 'Normalized_Importance' column exists
    if 'Normalized_Importance' not in feature_importance_results.columns:
        raise ValueError("The feature importance results must include a 'Normalized_Importance' column.")

    # Filter OTUs with non-zero Normalized Importance
    significant_otus = feature_importance_results[
        feature_importance_results['Normalized_Importance'] > 0
    ]

    # Map Normalized Importance to the abundance table columns
    importance_dict = significant_otus.set_index('OTU')['Normalized_Importance'].to_dict()

    # Ensure only relevant OTUs are used
    filtered_abundance_table = abundance_table[
        abundance_table.columns.intersection(importance_dict.keys())
    ]

    # Prepare values for calculations
    importance_values = filtered_abundance_table.columns.map(importance_dict)

    # Perform the specified operation
    if operation == '*':  # Multiplication
        bci_values = filtered_abundance_table.multiply(importance_values**amplify_factor, axis=1).sum(axis=1)
    elif operation == '2':  # Weighted sum of squares
        bci_values = (
            (filtered_abundance_table**2).multiply(importance_values**amplify_factor, axis=1).sum(axis=1)
        )
    elif operation == '3':  # Sum of weighted abundances
        bci_values = filtered_abundance_table.multiply(importance_values**amplify_factor, axis=1).sum(axis=1)
    elif operation == '4':  # Normalized importance scaled by abundance
        bci_values = filtered_abundance_table.div(importance_values**amplify_factor + 1e-8, axis=1).sum(axis=1)
    elif operation == '5':  # Sum of log-weighted contributions
        bci_values = np.log(filtered_abundance_table + 1).multiply(importance_values**amplify_factor, axis=1).sum(axis=1)
    else:
        raise ValueError("Invalid operation. Choose from '*', '2', '3', '4', '5'.")

    # Amplify the BCI values for better interpretability
    bci_values *= final_amplification

    # Create a DataFrame with the BCI values
    bci_df = pd.DataFrame(bci_values, columns=[f"{otu_type}_BCI"])
    bci_df.index.name = 'Sample_ID'

    return bci_df

In [9]:
bacterial_bci = bcon_index(reg_sev_16S, otu_table_16S, 'bacterial', operation='4')
fungal_bci = bcon_index(reg_sev_ITS, otu_table_ITS, 'fungal', operation='4')

In [10]:
def add_bci_to_data(data, bacterial_bci, fungal_bci):
    """
    Add bacterial_BCI and fungal_BCI columns to the data, prefixed with 'P_'.
    
    Parameters:
        data (pd.DataFrame): Original data with 'Sample_ID' as a column.
        bacterial_bci (pd.DataFrame): DataFrame containing bacterial_BCI values with Sample_ID as the index.
        fungal_bci (pd.DataFrame): DataFrame containing fungal_BCI values with Sample_ID as the index.
    
    Returns:
        pd.DataFrame: Updated data with P_bacterial_BCI and P_fungal_BCI columns.
    """
   
    # Merge bacterial BCI into the data
    data = data.merge(bacterial_bci, left_on='Sample_ID', right_index=True, how='left')
    
    # Merge fungal BCI into the data
    data = data.merge(fungal_bci, left_on='Sample_ID', right_index=True, how='left')
    
    return data

In [11]:
merged_data = add_bci_to_data(data, bacterial_bci, fungal_bci)

In [12]:
def duplicate_bci_scores(data, equivalence_table):
    """
    Fill missing BCI scores using equivalent samples from the equivalence table.

    Parameters:
        data (pd.DataFrame): Original data containing 'Sample_ID', 'Fungal_BCI', and 'Bacterial_BCI'.
        equivalence_table (pd.DataFrame): DataFrame with columns 'fungal_Sample_ID' and 'bacterial_Sample_ID'.

    Returns:
        pd.DataFrame: Updated data with BCI columns filled based on equivalence.
    """
    # Create mappings for equivalence
    fungal_to_bacterial = equivalence_table.set_index('fungal_Sample_ID')['bacterial_Sample_ID'].to_dict()
    bacterial_to_fungal = equivalence_table.set_index('bacterial_Sample_ID')['fungal_Sample_ID'].to_dict()

    # Fill missing Bacterial_BCI using bacterial equivalents
    for idx, row in data.iterrows():
        if pd.isna(row['bacterial_BCI']):
            sample_id = row['Sample_ID']
            if sample_id in fungal_to_bacterial:  # Check for fungal sample's equivalent bacterial sample
                equivalent_id = fungal_to_bacterial[sample_id]
                equivalent_row = data.loc[data['Sample_ID'] == equivalent_id]
                if not equivalent_row.empty:  # Ensure the equivalent bacterial row exists
                    data.at[idx, 'bacterial_BCI'] = equivalent_row['bacterial_BCI'].values[0]

    # Fill missing Fungal_BCI using fungal equivalents
    for idx, row in data.iterrows():
        if pd.isna(row['fungal_BCI']):
            sample_id = row['Sample_ID']
            if sample_id in bacterial_to_fungal:  # Check for bacterial sample's equivalent fungal sample
                equivalent_id = bacterial_to_fungal[sample_id]
                equivalent_row = data.loc[data['Sample_ID'] == equivalent_id]
                if not equivalent_row.empty:  # Ensure the equivalent fungal row exists
                    data.at[idx, 'fungal_BCI'] = equivalent_row['fungal_BCI'].values[0]

    return data

In [13]:
merged_data = duplicate_bci_scores(merged_data, equivalence_table)

In [14]:
def eco_metrics(abundance_table, otu_type, metric="braycurtis"):
    """
    Calculate ecological metrics (alpha diversity, evenness, and beta diversity) for microbial communities.

    Parameters:
        abundance_table (pd.DataFrame): Abundance table with OTUs as columns and Sample_ID as index.
        otu_type (str): Type of OTU ('fungal' or 'bacterial') to name the output columns.
        metric (str): Distance metric for beta diversity (default: 'braycurtis').

    Returns:
        pd.DataFrame: DataFrame with alpha diversity, beta diversity, and evenness as columns.
    """
    # Ensure no empty rows
    abundance_table = abundance_table.loc[abundance_table.sum(axis=1) > 0]
    
    # Alpha diversity (Shannon Index)
    alpha_div = abundance_table.apply(lambda row: shannon(row.values), axis=1)
    
    # Evenness calculation (Pielou's evenness)
    total_abundance = abundance_table.sum(axis=1)
    evenness = alpha_div / np.log(total_abundance.replace(0, 1))
    
    # Beta diversity (distance matrix)
    beta_div = beta_diversity(metric, abundance_table, ids=abundance_table.index)
    beta_div_mean = beta_div.to_data_frame().mean(axis=1)  # Calculate mean beta diversity for each sample

    # Construct the output DataFrame
    metrics_df = pd.DataFrame({
        f"{otu_type}_alpha_diversity": alpha_div,
        f"{otu_type}_evenness": evenness,
        f"{otu_type}_beta_diversity": beta_div_mean
    }, index=abundance_table.index)

    return metrics_df

In [15]:
bacterial_metrics = eco_metrics(otu_table_16S, otu_type='bacterial')
fungal_metrics = eco_metrics(otu_table_ITS, otu_type='fungal')

In [16]:
def add_metrics_to_data(data, bacterial_metrics, fungal_metrics):
    """
    Add bacterial and fungal ecological metrics to the data, with columns prefixed by 'P_'.
    
    Parameters:
        data (pd.DataFrame): Original data with 'Sample_ID' as a column.
        bacterial_metrics (pd.DataFrame): DataFrame containing bacterial metrics with Sample_ID as the index.
        fungal_metrics (pd.DataFrame): DataFrame containing fungal metrics with Sample_ID as the index.
    
    Returns:
        pd.DataFrame: Updated data with prefixed ecological metrics columns.
    """
    # Ensure prefix for bacterial metrics
    bacterial_metrics = bacterial_metrics.add_prefix("P_")
    
    # Ensure prefix for fungal metrics
    fungal_metrics = fungal_metrics.add_prefix("P_")
    
    # Merge bacterial metrics into the data
    data = data.merge(bacterial_metrics, left_on='Sample_ID', right_index=True, how='left')
    
    # Merge fungal metrics into the data
    data = data.merge(fungal_metrics, left_on='Sample_ID', right_index=True, how='left')
    
    return data

In [17]:
merged_data = add_metrics_to_data(merged_data, bacterial_metrics, fungal_metrics)

In [18]:
def duplicate_metrics(data, bacterial_metrics, fungal_metrics, equivalence_table):
    """
    Duplicate equivalent metrics to fill NaN values based on the equivalence table.

    Parameters:
        data (pd.DataFrame): Original data containing 'Sample_ID' as a column.
        bacterial_metrics (pd.DataFrame): DataFrame with bacterial metrics indexed by Sample_ID.
        fungal_metrics (pd.DataFrame): DataFrame with fungal metrics indexed by Sample_ID.
        equivalence_table (pd.DataFrame): DataFrame with columns 'fungal_Sample_ID' and 'bacterial_Sample_ID'.

    Returns:
        pd.DataFrame: Updated data with bacterial and fungal metrics filled based on equivalence.
    """
    # Merge bacterial and fungal metrics into the main data
    data = data.merge(bacterial_metrics, left_on='Sample_ID', right_index=True, how='left')
    data = data.merge(fungal_metrics, left_on='Sample_ID', right_index=True, how='left')

    # Create mappings from the equivalence table
    fungal_to_bacterial = equivalence_table.set_index('fungal_Sample_ID')['bacterial_Sample_ID'].to_dict()
    bacterial_to_fungal = equivalence_table.set_index('bacterial_Sample_ID')['fungal_Sample_ID'].to_dict()

    # Fill missing fungal metrics using only fungal metrics from equivalent fungal samples
    for idx, row in data.iterrows():
        if pd.isna(row['fungal_alpha_diversity']):  # Check if fungal metric is missing
            sample_id = row['Sample_ID']
            if sample_id in bacterial_to_fungal:  # Find bacterial equivalent
                equivalent_id = bacterial_to_fungal[sample_id]
                equivalent_row = data.loc[data['Sample_ID'] == equivalent_id]
                if not equivalent_row.empty:  # Ensure the equivalent row exists
                    # Copy only fungal metrics
                    for col in data.filter(like="fungal_").columns:
                        data.at[idx, col] = equivalent_row[col].values[0]

    # Fill missing bacterial metrics using only bacterial metrics from equivalent bacterial samples
    for idx, row in data.iterrows():
        if pd.isna(row['bacterial_alpha_diversity']):  # Check if bacterial metric is missing
            sample_id = row['Sample_ID']
            if sample_id in fungal_to_bacterial:  # Find fungal equivalent
                equivalent_id = fungal_to_bacterial[sample_id]
                equivalent_row = data.loc[data['Sample_ID'] == equivalent_id]
                if not equivalent_row.empty:  # Ensure the equivalent row exists
                    # Copy only bacterial metrics
                    for col in data.filter(like="bacterial_").columns:
                        data.at[idx, col] = equivalent_row[col].values[0]

    return data

In [19]:
merged_data = duplicate_metrics(merged_data, bacterial_metrics, fungal_metrics, equivalence_table)

In [20]:
def add_p_prefix(data):
    """
    Add the prefix 'P_' to column names in the DataFrame that start with 'fungal' or 'bacterial'
    and do not already start with 'P_'.

    Parameters:
        data (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with updated column names.
    """
    # Update column names
    data = data.rename(
        columns=lambda col: f"P_{col}" if col.startswith(('fungal', 'bacterial')) and not col.startswith('P_') else col
    )
    return data

In [21]:
merged_data = add_p_prefix(merged_data)
merged_data.set_index(merged_data.columns[0], inplace=True)

In [22]:
def remove_duplicate_samples(data, predictor_columns):
    """
    Removes one occurrence of rows with duplicate values across all predictor columns.

    Parameters:
    - data (pd.DataFrame): The input DataFrame containing predictors and other columns.
    - predictor_columns (list): A list of predictor column names to check for duplicates.

    Returns:
    - pd.DataFrame: A DataFrame with duplicates removed but one instance retained.
    """
    # Remove duplicates, keeping the first occurrence
    filtered_data = data[~data.duplicated(subset=predictor_columns, keep='first')]
    return filtered_data

In [23]:
# Remove exact duplicate columns
merged_data = merged_data.loc[:, ~merged_data.columns.duplicated()]

In [24]:
# Create the new columns by calculating the square root of the sum of squares of bacterial and fungal columns
merged_data.loc[:, 'P_BCI'] = merged_data['P_bacterial_BCI']**2 + merged_data['P_fungal_BCI']**2
merged_data.loc[:, 'P_alpha_diversity'] = (merged_data['P_bacterial_alpha_diversity']**2 + merged_data['P_fungal_alpha_diversity']**2)**0.5
merged_data.loc[:, 'P_evenness'] = (merged_data['P_bacterial_evenness']**2 + merged_data['P_fungal_evenness']**2)**0.5
merged_data.loc[:, 'P_beta_diversity'] = (merged_data['P_bacterial_beta_diversity']**2 + merged_data['P_fungal_beta_diversity']**2)**0.5

# Drop the original bacterial and fungal columns
merged_data = merged_data.drop(columns=[
    'P_bacterial_BCI', 'P_fungal_BCI',
    'P_bacterial_alpha_diversity', 'P_fungal_alpha_diversity',
    'P_bacterial_evenness', 'P_fungal_evenness',
    'P_bacterial_beta_diversity', 'P_fungal_beta_diversity'
])

In [25]:
classifiers = [col for col in merged_data.columns if col.startswith('K_')]
predictors = [col for col in merged_data.columns if col.startswith('P_')]
target = [col for col in merged_data.columns if col.startswith('Y_')]

In [26]:
merged_data = remove_duplicate_samples(merged_data, predictors)

In [27]:
classifiers = [col for col in merged_data.columns if col.startswith('K_')]
predictors = [col for col in merged_data.columns if col.startswith('P_')]
target = [col for col in merged_data.columns if col.startswith('Y_')]

In [28]:
# Impute missing values for predictors
imputer = KNNImputer(n_neighbors=5)
merged_data[predictors] = imputer.fit_transform(merged_data[predictors])

In [29]:
# Scale predictors
scaler = StandardScaler()
merged_data[predictors] = scaler.fit_transform(merged_data[predictors])
#merged_data[target] = scaler.fit_transform(merged_data[target])

In [30]:
# response midpoint
min_value = merged_data['Y_rs.dis'].min()
max_value = merged_data['Y_rs.dis'].max()

midpoint = (min_value + max_value) / 2

print(f"The midpoint of Y_rs.dis is: {midpoint}")

The midpoint of Y_rs.dis is: 57.142857145


In [31]:
# Calculate descriptive metrics
def calculate_descriptive_metrics(data, otu_table_16S, otu_table_ITS):
    descriptive_metrics = {
        "Dataset": ["Main Data", "16S OTUs", "ITS OTUs"],
        "Number of Variables": [data.shape[1], otu_table_16S.shape[1], otu_table_ITS.shape[1]],
        "Number of Samples": [data.shape[0], otu_table_16S.shape[0], otu_table_ITS.shape[0]],
        "Number of OTUs": [None, otu_table_16S.shape[1], otu_table_ITS.shape[1]],
    }
    return pd.DataFrame(descriptive_metrics)

# Generate the descriptive metrics table
descriptive_metrics_table = calculate_descriptive_metrics(data, otu_table_16S, otu_table_ITS)

# Display the descriptive metrics
print(descriptive_metrics_table)

     Dataset  Number of Variables  Number of Samples  Number of OTUs
0  Main Data                   22                160             NaN
1   16S OTUs                14652                 80         14652.0
2   ITS OTUs                 1402                 80          1402.0


In [32]:
# Save the DataFrame as a CSV file
output_path = '/Users/vbuttros/Library/CloudStorage/OneDrive-Personal/University/Microbiology - DSc/Colaborations/Buttros, VH/R_solanii_ANN/Treated_data/dataset2_script_files/merged_data.csv'
merged_data.to_csv(output_path, index=False)